<a href="https://colab.research.google.com/github/DongHyunByun/doc2vec_similarity_test/blob/master/model_by_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [ ]:
 class Doc2VecSentences:
    def __init__(self,path):
        self.path=path
        self.keyDict={}
        
    def clean_str(self,text):
        pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
        text = re.sub(pattern=pattern, repl='', string=text)
        pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
        text = re.sub(pattern=pattern, repl='', string=text)
        pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
        text = re.sub(pattern=pattern, repl='', string=text)
        pattern = '<[^>]*>'         # HTML 태그 제거
        text = re.sub(pattern=pattern, repl='', string=text)
        pattern = '[^\w\s]'         # 특수기호제거
        text = re.sub(pattern=pattern, repl='', string=text)
        return text
    
    def __iter__(self):
        # 파일을 다시 처음부터 읽음.
        result = []
        keyId=0
        fread = open(self.path, encoding="utf8")
        print("!")
        while (1):
            line = fread.readline() #한 줄씩 읽음.
            if not line: 
                break # 모두 읽으면 while문 종료.
            lineL=line.split()
            if lineL:
                #새문서 시작
                if lineL[0]=="</doc>":
                    fread.readline()
                    title=fread.readline().rstrip("\n")
                    self.keyDict[keyId]=title
                    keyId+=1
                elif lineL[0]=='<doc':
                    title=fread.readline().rstrip("\n")
                    self.keyDict[keyId]=title
                    keyId+=1
                #그냥문장
                else:
                    lineL=list(map(self.clean_str,lineL))
                    yield TaggedDocument(words=lineL, tags=['#%d'%keyId])

        fread.close()


In [ ]:
path='wiki_data.txt'
doc2vec_corpus=Doc2VecSentences(path)
print("?")
doc2vec_model = Doc2Vec(doc2vec_corpus)